## Проект по предсказанию стоимости автомобиля на вторичном рынке.

Целью проекта будет разработать модель предсказания стоимости автомобиля на вторичном рынке. (целевая метрика - MAPE)

### Импорт билиотек

In [ ]:
# импорт основных библиотек для работы с данными и визуализации данных
import pandas as pd
import numpy as np
import re
from time import strptime
import matplotlib.pyplot as plt
import seaborn as sns
      
# пердобработка
from statsmodels.stats.outliers_influence import variance_inflation_factor  
from statsmodels.tools.tools import add_constant  

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import RobustScaler

from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from joblib import Parallel, delayed

# отбор данных
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# модели, которые будут тестироваться
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor

# метрики
from sklearn.metrics import make_scorer, mean_absolute_percentage_error

# рапспределения
from scipy.stats import shapiro
from scipy import stats

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

### Загрузка и ознакомление с данными

In [ ]:
# загрузим данные
df_train = pd.read_csv('../input/car-prices/train.csv') 
df_test = pd.read_csv('../input/car-prices/test.csv')

valid_data = df_train.copy() # используется в конце

In [ ]:
# выведим информацию о типах данных
df_train.info(), print('\n'), df_test.info();

In [ ]:
# изучим первые 5 записей
df_train.sample(frac=1).head(5)

In [ ]:
df_test.sample(frac=1).head(5)

In [ ]:
# проверим есть ли записе о повторной продаже одного и того же авто
print('Все инн в тренировочных данных уникальные:', len(set(df_train['vin'])) == df_train.shape[0])
print('Все инн в тестовых данных уникальные:', len(set(df_test['vin'])) == df_test.shape[0])

### Предварительная обработка

In [ ]:
def preprocessing(df):
    
    '''
    Функция  выполняет поэтапно все процессы предобработки данных
    '''
    
    def date_creating(df):
        '''
        Функция разделяет дату и возвращает фрейм с датой в формате (месяц, число, год)
        '''
        list_of_dates = []
        # циклом пройдёмся по датам
        for mean in df['saledate']:
            list_of_date_objects = mean.split(' ') # каждую дату разделим на список
            month_number = [str(strptime(list_of_date_objects[1],'%b').tm_mon)] # месяц в его номер, задав тип - str
            date = '-'.join(month_number+list_of_date_objects[2:4]) # к месяцу добавим дату и месяц, и избавимся от списка
            list_of_dates.append(date)

        df['saledate'] = list_of_dates # замени дату на новую
        df['saledate'] = pd.to_datetime(df['saledate'], format='%m-%d-%Y') # переведём в нужный формат
        return df
    
    def clean_text(df):
        '''
        Функция приводит текст к нижнему регистру, избавляет от всех лишних знаков. Оставляет только буквы и цифры
        '''
        for column in df.columns:
            # только для текстовых значений
            if df[column].dtype == ('object'):
                # переведём всё в нижний регистр
                df[column] = df[column].str.lower()
                # оставим только буквы
                df[column] = df[column].str.replace('[^\w\s]+', '')
                df[column] = df[column].replace('', np.nan)
        return df
    
    
    def drop_missing_values(df):
        '''
        Функция избавляется от пропусков:
            1. для числовых признаков - подставляет медиану
            2. для категориальных признаков подставляет unknown.
        Решение обосновано незначительным количеством пропусков в данных
        '''
        # пройдёмся по каждому столбцу
        for column in df.columns:

            # если столбец числовой и пропуски в нём есть, заменим данные на медианное значение
            if (df[column].dtype == ('int64') or df[column].dtype == ('float64')) and len(df[~df[column].notna()]) > 0:
                df[column]=df[column].replace(np.nan, df[column].median())

            # в обратной ситуации, заменим пропуски методом 
            else:
                df[column] = df[column].fillna('unknown')
        return df
    
    # удаляем строки с inf значениями
    df = df[~df.isin([np.inf, -np.inf]).any(1)]
    
    # создаём дату
    df = date_creating(df)
    
    # удаляем строки с отсутствующими значениями
    df = drop_missing_values(df)

    # возвращаем предобработанный датафрейм
    return df

In [ ]:
df_train = preprocessing(df_train)
df_test = preprocessing(df_test)

### Разведочный анализ

In [ ]:
df_train.describe()

In [ ]:
df_train.boxplot()
plt.show()

In [ ]:
# также проверим распределение на нормальное для целевой переменной
# применим критерий Шапиро-Уилка
stat, p = shapiro(df_train['sellingprice'])
print('stat=%.3f, p=%.3f' % (stat, p))

# интерпретируем результаты
alpha = 0.05
if p > alpha:
    print('Распределение похоже на нормальное (не отвергаем H0)')
else:
    print('Распределение не похоже на нормальное (отвергаем H0)')

**Наблюдаются явные выбросы по пробегу и цене автомобилей. Избавимся от них**

In [ ]:
def remove_outliers_iqr(df, columns=None, multiplier=1.5):
    """
    Функция для удаления статистических выбросов из датафрейма с использованием метода IQR
    
    Аргументы:
    df - исходный датафрейм
    columns - список столбцов для обработки. Если None, то будут обработаны все числовые столбцы
    multiplier - коэффициент множителя межквартильного расстояния
    
    Возвращает:
    Новый датафрейм без статистических выбросов
    """
    if columns is None:
        columns = df.select_dtypes(include=np.number).columns.tolist()
    
    df_out = df.copy()
    
    for col in columns:
        q1, q3 = np.percentile(df_out[col], [25, 75])
        iqr = q3 - q1
        lower_bound = q1 - (multiplier * iqr)
        upper_bound = q3 + (multiplier * iqr)
        df_out = df_out.loc[(df_out[col] >= lower_bound) & (df_out[col] <= upper_bound)]
    
    return df_out

df_train = remove_outliers_iqr(df_train)

### Разработка новых синтетических признаков и проверка на мультиколлинеарность. Отбор признаков

In [ ]:
def creating_features(df):
    # выделим год и месяц
    df['year_sale'] = df['saledate'].dt.year
    df['month_sale'] = df['saledate'].dt.month

    # распарсим вин код
    df['country'] = [mean[0] for mean in df['vin']] 
    df['manufacturer'] = [mean[1] for mean in df['vin']]
    df['security'] = [mean[4] for mean in df['vin']]
    df['max_weight'] = [mean[5] for mean in df['vin']]
    df['engine_type'] = [mean[6] for mean in df['vin']]
    df['transmission'] = [mean[11] for mean in df['vin']]
    df['dop_attributes'] = [mean[12] for mean in df['vin']]
    df['color'] = [mean[13] for mean in df['vin']]
    df['emission'] = [mean[16] for mean in df['vin']]

    # создадим признак сезон продажи
    df['season'] = df['month_sale'].apply(lambda x: 'winter' if x in [12, 1, 2] else 'spring' if x in [3, 4, 5] else 'summer' if x in [6, 7, 8] else 'autumn')
    
    # возраст авто
    df['age'] = df['year_sale'] - df['year']
    df.loc[df['age'] == 0, 'age'] = 1e-6

    # создание нового признака - средний год пробега
    df['avg_odometer_year'] = df['odometer'] / df['age']
    df.loc[df['avg_odometer_year'] == float('inf'), 'avg_odometer_year'] = 1e-6

    # создание нового признака - количество лет выпуска модели
    df['model_age'] = df['year_sale'] - df.groupby(['make', 'model'])['year_sale'].transform('min')
    
    return df

df_train = creating_features(df_train)
df_test = creating_features(df_test)

In [ ]:
df_train = df_train.drop('saledate', axis=1)
df_test = df_test.drop('saledate', axis=1)

In [ ]:
# заменить все значения "inf" на NaN
df_train = df_train.replace([np.inf, -np.inf], np.nan) 

# удалить все строки, содержащие NaN
df_train = df_train.dropna()

In [ ]:
target = df_train['sellingprice'].copy()
features = df_train.drop('sellingprice', axis=1).copy()

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.2, random_state=12345, shuffle=False)

features_test = df_test.copy()

num_cols = selector(dtype_exclude=[object, bool]) # опредлим числовые объекты
cat_cols = selector(dtype_include=object)         # опредлим категориальные объекты

categorical_oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=999999)

numerical_rs = RobustScaler()

preprocessor_oe_rs = ColumnTransformer([
('oe', categorical_oe, cat_cols),
('rs', numerical_rs, num_cols)])

new_columns_names = list(features.columns.values)

preprocessor_oe_rs.fit(features_train)
features_train_oe_rs = pd.DataFrame(preprocessor_oe_rs.transform(features_train), columns=new_columns_names)
features_valid_oe_rs = pd.DataFrame(preprocessor_oe_rs.transform(features_valid), columns=new_columns_names)
features_test_oe_rs = pd.DataFrame(preprocessor_oe_rs.transform(features_test), columns=new_columns_names)

In [ ]:
def remove_multicollinearity(df_train, df_valid, df_test, threshold=5):
    """
    Функция для удаления мультиколлинеарных признаков из дата-фрейма
    
    Аргументы:
    df_train - дата-фрейм с тренировочными данными
    df_valid - дата-фрейм с валидационными данными
    df_test - дата-фрейм с тестовыми данными
    threshold - пороговое значение VIF
    
    Возвращает:
    Тройку новых дата-фреймов, где удалены мультиколлинеарные признаки
    """
    # Объединяем все фреймы в один
    df_all = pd.concat([df_train, df_valid, df_test], axis=0)
    X = (df_all - df_all.mean()) / df_all.std()
    X = np.nan_to_num(X)
    vif = Parallel(n_jobs=-1)(delayed(variance_inflation_factor)(X, i) for i in range(X.shape[1]))
    
    # Находим мультиколлинеарные признаки для всего объединенного фрейма
    multicollinear_features = [df_all.columns[i] for i, v in enumerate(vif) if v > threshold]
    print('Multicollinear features:', multicollinear_features)
    
    # Удаляем мультиколлинеарные признаки из всех трех фреймов
    df_train = df_train.drop(multicollinear_features, axis=1)
    df_valid = df_valid.drop(multicollinear_features, axis=1)
    df_test = df_test.drop(multicollinear_features, axis=1)
    
    return df_train, df_valid, df_test


df_train, df_valid, df_test = remove_multicollinearity(features_train_oe_rs, features_valid_oe_rs, features_test_oe_rs) 

In [ ]:
# дополнительно проверим сработал ли код выше, представим признаки в виде матрицы:
corr_matrix = df_train.corr()

# визуализация матрицы корреляций с помощью тепловой карты
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, square=True)
plt.figure(figsize=(15, 25))
plt.show()

In [ ]:
train_features = pd.concat([df_train, df_valid], axis=0).reset_index(drop=True)
train_target = pd.concat([target_train, target_valid], axis=0).reset_index(drop=True)
test_features = df_test.copy()

### Выбор и обучение моделей

In [ ]:
xgb_params = {'n_estimators': [100, 200, 300],
              'max_depth': [3, 5, 7],
              'learning_rate': [0.01, 0.1, 0.3]}

rf_params = {'n_estimators': [100, 200, 300],
             'max_depth': [3, 5, 7],
             'min_samples_split': [2, 5, 10]}

gb_params = {'n_estimators': [100, 200, 300],
             'max_depth': [3, 5, 7],
             'learning_rate': [0.01, 0.1, 0.3]}

sgd_params = {'max_iter': [1000, 5000, 10000],
              'alpha': [0.0001, 0.001, 0.01],
              'penalty': ['l1', 'l2']}

models = [
    {'name': 'XGBRegressor', 'model': XGBRegressor(), 'params': xgb_params},
    {'name': 'RandomForestRegressor', 'model': RandomForestRegressor(), 'params': rf_params},
    {'name': 'GradientBoostingRegressor', 'model': GradientBoostingRegressor(), 'params': gb_params},
    {'name': 'SGDRegressor', 'model': SGDRegressor(), 'params': sgd_params}
]

# перевернём метрику сделав её обратной;
scoring_metric = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# инициализируем результаты лучшей модели
best_model = None
best_score = float('inf')

# циклом подберём гиперпараметры
for model in models:
    print('Обучние', model['name'], '...')
    clf = GridSearchCV(model['model'], model['params'], scoring=scoring_metric, cv=4, n_jobs=-1, verbose=2)
    clf.fit(train_features.loc[:1000], train_target.loc[:1000])
    score = clf.best_score_
    print(model['name'], '- Лучший результат:', score)
    print(model['name'], '- Лучшие параметры:', clf.best_estimator_)
    if abs(score) < abs(best_score):
        best_score = score
        best_model = clf.best_estimator_

print('Лучшая модель:', best_model)

In [ ]:
# Задаем гиперпараметры модели
xgb_params_v2 = {'n_estimators': [100, 300, 500],
                 'max_depth': [3, 5, 7],
                 'learning_rate': [0.01, 0.1, 0.3, 0.5],
                 'subsample': [0.5, 0.7, 0.9],
                 'reg_alpha': [0, 1],
                 'reg_lambda': [0, 1]}

# Создаем модель XGBRegressor
xgb = XGBRegressor()

# Подбираем лучшие гиперпараметры
clf_v2 = GridSearchCV(xgb, xgb_params_v2, scoring=scoring_metric, cv=4, n_jobs=-1)
clf_v2.fit(train_features, train_target, eval_metric='mape', eval_set=[(train_features, train_target)])

# Выбираем лучшую модель

print(clf_v2.best_score_)
best_model_xgb = clf_v2.best_estimator_

In [ ]:
best_model_xgb.fit(train_features, train_target)
preds = best_model_xgb.predict(test_features)

res = pd.DataFrame({'vin': [mean for mean in valid_data['vin']],
                    'sellingprice': preds})

res.to_csv("../input/car-prices/finally.csv", index=False)

### Вывод

Резюмирущее качество модели на тестовой выборке (public) составило 10% MAPE. На тестовой выборке (private) - 25%.
Основные слабые стороны проекта, улучшив которые, возможно улучшить и результаты валидации:
    
    1) Улучшить предобработку;
    2) Обработать каждый признак, избавиться от неявных дубликатов;
    3) Протестировать модели, основанные на нейросетях;
    4) Попробовать найти дополнительные дата-фреймы с схожей структрой для дообучения модели;
    5) Гиперпараметры моделей выбирались на маленькой выборке данных, исходя из ограничений по временным ресурам.

Основные сильные стороны проекта, которые позволили обеспечить соотвествующую MAPE:

    1) Удалось избавиться от пропусков, без удаления данных;
    2) Статистические выбросы были очищенны с помощью умной системы, адаптирующей к каждому признаку;
    3) Полностью изменив данные в тренировочной выборке, модель всё равно выполнит работу;
    4) Создано много признаков, исследована мультиколенниарность;
    5) Осуществлялся большой объём подбора гиперараметров.
    
Mape в 25% - не самый лучший результат, но модель частично готова к проектированию под поток данных (поскольку код прописан в виде функций, то может выполняться независимо от состава признаков). 